In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rabieelkharoua/alzheimers-disease-dataset")

print("Path to dataset files:", path)

# https://www.kaggle.com/code/adhamtarek147/alzheimer-s-disease-prediction
# Eksempel av løsning på datasettet

In [18]:
import pandas as pd
data = pd.read_csv(path + "/alzheimers_disease_data.csv")
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,0,0,0.014691,0,0,1,1,0,0,XXXConfid


In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
#som beskrevet i datasett sin beskrivelse  er doctorInCharge bare XXXconfid og dermed ikke relevant
data_cleaned = data.drop(columns=["DoctorInCharge"], axis=1)
#vi trenger heller ikkje PatientID
data_cleaned = data_cleaned.drop(columns=["PatientID"], axis=1)
data_cleaned.info()

In [ ]:
# Siden dette bare en enkel nettside så blir det vanskielig å få tilgang til medisisnke verdiser som CholesterolHDL og lignende.
# Så dermed fjerner vi disse kolonnene siden vi ikkje kan forvente brukeren å kunne disse verdiene.
data_cleaned = data_cleaned.drop(columns=["CholesterolTotal","CholesterolLDL","CholesterolHDL","CholesterolTriglycerides","MMSE","FunctionalAssessment","ADL"], axis=1)
data_cleaned.info()

In [ ]:
#

In [ ]:
for column in data_cleaned.columns:
    unique_values = data_cleaned[column].unique()
    print(f"Column '{column}' has {unique_values}.")

# vi ser her at et veldig stor foskjell mellom skaleringen til nokon av verdiene med, tall fra 0-3 og nokon fra 0-100
# vi må normalisere de

In [ ]:
from sklearn.preprocessing import StandardScaler

columns_to_normalize = ['Age','BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP','DiastolicBP',
                        '','','','','MMSE','FunctionalAssessment','ADL']

# skalere dataen
standard_scaler = StandardScaler()
data_cleaned[columns_to_normalize] = standard_scaler.fit_transform(data_cleaned[columns_to_normalize])
data_cleaned.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = data_cleaned.drop(columns=["Diagnosis"], axis=1)
y = data_cleaned["Diagnosis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# vi tester flere modeler samtidig for å finne den beste
# vi tester da gjerne modeler som har en predict_proba metode siden alzheimers som kan være skadelig å gi konkrette spådommer om
# så vil bare gi en sannsynlighet for å ha sjukdommen og en anbefaling om å ta vidare tester

